In [1]:
import sys
sys.path.insert(1, ".")

from Utils import Utils
import numpy as np

In [2]:
vocab = Utils.file_to_vocab("./data/en-train.txt")

In [3]:
# simple baseline
#Utils.gen_baseline_labels("./data/en-test.txt", "./simple_baseline_test.txt", vocab)

In [4]:
train_Y = Utils.extract_labels('./data/en-train.txt')


In [5]:
# generate cosine feature on training data and save
with open('./data/en-train.txt','r') as f1:
    lines = f1.readlines()

# extract cosine feature
cosine_sim_feature = Utils.gen_cosine_sim_feature(lines, vocab)
train_X = np.array(cosine_sim_feature)
pickle.dump(train_X,open("feature_train_cosine.pickle","wb"))

0 3.5355339059327378
1000 1.571348402636772
2000 0.5330017908890261
3000 2.5
4000 0.8838834764831843
5000 3.5355339059327373
6000 3.4020690871988584
7000 1.875
8000 4.0
9000 4.444444444444445
10000 2.8571428571428568
11000 1.666666666666667
12000 1.3363062095621219
13000 2.004459314343183


NameError: name 'pickle' is not defined

In [6]:
from sklearn.kernel_ridge import KernelRidge

In [ ]:
clf = KernelRidge(alpha=1.0)
clf.fit(X, Y) 

In [7]:
float('3.9')

3.9